In [ ]:
# 이미지 파일 읽기
import numpy as np
import cv2
'''
cv2.IMREAD_COLOR : Loads a color image. Any transparency of image will be neglected. It is the default flag.
cv2.IMREAD_GRAYSCALE : Loads image in grayscale mode
cv2.IMREAD_UNCHANGED : Loads image as such including alpha chann
'''

# Load an color image in grayscale
img = cv2.imread('img/snoopy.jpg',0)
cv2.imshow('img/image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# image_pyramids
'''
동일 이미지의 서로 다른 사이즈의 set을 Image Pyramids
(가장 아래에 가장 큰 해상도를 놓고 점점 줄여가면서 쌓아가는 형태입니다.)
'''
import cv2

img = cv2.imread('img/lena.jpg')


lower_reso = cv2.pyrDown(img) # 원본 이미지의 1/4 사이즈
higher_reso = cv2.pyrUp(img) #원본 이미지의 4배 사이즈

cv2.imshow('img', img)
cv2.imshow('lower', lower_reso)
cv2.imshow('higher', higher_reso)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
# image_pyramids2
import cv2
import numpy as np

img = cv2.imread('img/lena.jpg')
print(img.shape)
GAD = cv2.pyrDown(img)
print(GAD.shape)
GAU = cv2.pyrUp(GAD)
print(GAU.shape)
temp = cv2.resize(GAU, (400, 225))
res = cv2.subtract(img, temp)
cv2.imshow('res',res) #외곽선만 남음
cv2.waitKey(0)

In [ ]:
# feature_matching_knnMatch
'''
Brute-Force Matching with SIFT Descriptors and Ratio Test
This time, we will use BFMatcher.knnMatch() to get k best matches
'''
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('img/lena.jpg',0)      # queryImage
img2 = cv2.imread('img/lena_face.png',0) # trainImage
img3 = None

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])

# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,img3,flags=2)

plt.imshow(img3),plt.show()

In [ ]:
# fourier_transform
'''
푸리에 변환 주파수를 분석하는데 사용되는 방법
주파수는 시간의 흐름에 따른 진동하는 파동의 횟수를 의미
이미지도 파동으로 변환을 할 수가 있는데, 주변 픽셀과의 밝기 변환가 많은 곳은 고주파로, 변환이 적은 곳은 저주파로 표현이 가능
즉, 이미지에서 고주파의 의미는 경계선을 의미하고, 저주파는 배경을 의미
그러므로 고주파를 제거하면 경계선이 사라지고, 저주파를 제거하면 경계선만 남음
이미지 -> 푸리에 변환 -> 고주파 또는 저주파 제거 -> 
다시 이미지 변환 과정을 거쳐 경계 또는 배경만 남음
'''
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('img/lena.jpg',0)
dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)

dft_shift = np.fft.fftshift(dft)
magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))

rows, cols = img.shape
crow,ccol = rows/2 , cols/2

# 아래는 d 사이지의 사각형을 생성한 후, 사각형 바깥쪽을 제거하는 형태임.
# 즉, 고주파영역을 제거하게 됨.
# d값이 작을수록 사각형이 작고, 바깥영역 즉, 고주파영역이  많이 제거되기 때문에 이미지가 뭉게지고
# d값이 클수록 사각형이 크고, 바깥영역 즉, 고주파 영역이 적게 제거되기 때문에 원래 이미지와 가까워짐.

d = 30
mask = np.zeros((rows,cols,2),np.uint8)
print(d)
mask[int(crow-d):int(crow+d), int(ccol-d):int(ccol+d)] = 1
# apply mask and inverse DFT
fshift = dft_shift*mask
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])

plt.subplot(121),plt.imshow(img, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img_back, cmap = 'gray')
plt.title('FT'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# mask_histogram
# 이미지의 특정 영역의 히스토그램을 분석하기 위해서 mask를 적용
import cv2
import numpy as np
from matplotlib import pyplot as plt


img = cv2.imread('img/lena.jpg')

# mask생성
# create a mask image with white color on the region you want to find histogram and black otherwise. 
# Then pass this as the mask.
mask = np.zeros(img.shape[:2],np.uint8)
mask[100:300,100:400] = 255

# 이미지에 mask가 적용된 결과
masked_img = cv2.bitwise_and(img,img,mask=mask)

# 원본 이미지의 히스토그램
hist_full = cv2.calcHist([img],[1],None,[256],[0,256])

# mask를 적용한 히스트로그램
hist_mask = cv2.calcHist([img],[1],mask,[256],[0,256])

plt.subplot(221),plt.imshow(img,'gray'),plt.title('Origianl Image')
plt.subplot(222),plt.imshow(mask,'gray'),plt.title('Mask')
plt.subplot(223),plt.imshow(masked_img,'gray'),plt.title('Masked Image')

# red는 원본이미지 히스토그램, blue는 mask적용된 히스토그램
plt.subplot(224),plt.title('Histogram')
plt.plot(hist_full,color='r'),plt.plot(hist_mask,color='b')
plt.xlim([0,256])

plt.show()

In [ ]:
# brute-force_matchingwithORB_descriptors
'''
이미지의 특징점 매칭
ORB 기술자를 사용한 특징점 비교
'''
import numpy as np
import cv2
from matplotlib import pyplot as plt
#plt.figure(figsize=(10,10))

img1 = cv2.imread('img/lena.jpg',0)
img2 = cv2.imread('img/lena_face.png',0)
res = None

orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)

matches = sorted(matches, key=lambda x:x.distance)
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:10],None,flags=2)

cv2.imshow('img',img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 2d_histogram
# Histogram은 1차원으로 grayscale 이미지의 pixel의 강도, 즉 빛의 세기를 분석한 결과였습니다. 
# 2D Histogrm은 Color 이미지의 Hue(색상) & Saturation(채도)을 동시에 분석하는 방법
'''
calcHist([image, ][channel, ]mask[, bins][, range])
Histogram 분석 함수
Parameters:	
image – HSV로 변환된 이미지
channel – 0-> Hue, 1-> Saturation
bins – [180,256] 첫번째는 Hue, 두번째는 Saturation
range – [0,180,0,256] : Hue(0~180), Saturation(0,256)
'''
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/hist_test3.jpg')
hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

hist = cv2.calcHist([hsv],[0,1],None,[180,256],[0,180,0,256])

cv2.imshow('Hist',hist)

plt.imshow(hist) #, interpolation='nearest')
plt.show() # Histogram을 보면 X축은 Saturation, Y축은 Hue값을 나타냅니다

In [1]:
# 이미지 파일 합치기
import cv2
import numpy as np

img1 = cv2.imread('img/img1.png')
img2 = cv2.imread('img/img2.png')

dst = cv2.addWeighted(img1,0.7,img2,0.3,0)

cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
# 이미지 파일 합치기2 image_blending_using_pyramids
import cv2
import numpy as np
from matplotlib import pyplot as plt

A = cv2.imread('img/apple.png')
B = cv2.imread('img/orange.png')

# A 이미지에 대한 Gaussian Pyramid를 생성
# 점점 작아지는 Pyramid
G = A.copy()
gpA = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpA.append(G)

# B 이미지에 대한 Gaussian Pyramid 생성
# 점점 작아지는 Pyramid
G = B.copy()
gpB = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpB.append(G)

# A 이미지에 대한 Laplacian Pyramid 생성
lpA = [gpA[5]] # n번째 추가된 Gaussian Image
for i in range(5,0,-1):
    GE = cv2.pyrUp(gpA[i]) #n번째 추가된 Gaussian Image를 Up Scale함.
    temp = cv2.resize(gpA[i-1], (GE.shape[:2][1], GE.shape[:2][0])) # 행렬의 크기를 동일하게 만듬.
    L = cv2.subtract(temp,GE) # n-1번째 이미지에서 n번째 Up Sacle한 이미지 차이 -> Laplacian Pyramid
    lpA.append(L)

# A 이미지와 동일하게 B 이미지도 Laplacian Pyramid 생성
lpB = [gpB[5]]
for i in range(5,0,-1):
    GE = cv2.pyrUp(gpB[i])
    temp = cv2.resize(gpB[i - 1], (GE.shape[:2][1], GE.shape[:2][0]))
    L = cv2.subtract(temp, GE)
    # L = cv2.subtract(gpB[i-1],GE)
    lpB.append(L)

# Laplician Pyramid를 누적으로 좌측과 우측으로 재결함
LS = []
for la,lb in zip(lpA,lpB):
    rows,cols,dpt = la.shape

    ls = np.hstack((la[:,0:int(cols/2)], lb[:,int(cols/2):]))
    LS.append(ls)

ls_ = LS[0] # 좌측과 우측이 결합된 가장 작은 이미지
for i in range(1,6):
    ls_ = cv2.pyrUp(ls_) # Up Sacle
    temp = cv2.resize(LS[i],(ls_.shape[:2][1], ls_.shape[:2][0])) # 외곽선만 있는 이미지
    ls_ = cv2.add(ls_, temp) # UP Sacle된 이미지에 외곽선을 추가하여 선명한 이미지로 생성

# 원본 이미지를 그대로 붙인 경우
real = np.hstack((A[:,:int(cols/2)],B[:,int(cols/2):]))

cv2.imshow('real', real)
cv2.imshow('blending', ls_)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
'''
hough_tranform 
허프 변환을 이용하여 이미지의 Line을 찾음
이미지의 형태를 찾거나, 누락되거나 깨진 영역을 복원
cv2.HoughLines(image, rho, theta, threshold[, lines[, srn[, stn[, min_theta[, max_theta]]]]]) → lines
Parameters:	
image – 8bit, single-channel binary image, canny edge를 선 적용.
rho – r 값의 범위 (0 ~ 1 실수)
theta – 𝜃 값의 범위(0 ~ 180 정수)
threshold – 만나는 점의 기준, 숫자가 작으면 많은 선이 검출되지만 정확도가 떨어지고, 숫자가 크면 정확도가 올라감.
'''
import cv2
import numpy as np

img = cv2.imread('img/board.jpg')
img_original = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize=3)

lines = cv2.HoughLines(edges,1,np.pi/180,100)

for i in range(len(lines)):
    for rho, theta in lines[i]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0+1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 -1000*(a))

        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

res = np.vstack((img_original,img))
cv2.imshow('img',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# image_blurring
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/tree.jpg')

# pyplot를 사용하기 위해서 BGR을 RGB로 변환.
b,g,r = cv2.split(img)
img = cv2.merge([r,g,b])


# 일반 Blur
# Box형태의 kernel을 이미지에 적용한 후 평균값을 box의 중심점에 적용하는 형태
dst1 = cv2.blur(img,(7,7))

# GaussianBlur
# kernel의 사이즈는 양수이면서 홀수로 지정을 해야 합니다. 
# 이미지의 Gaussian Noise (전체적으로 밀도가 동일한 노이즈, 백색노이즈)를 제거하는 데 가장 효과적
dst2 = cv2.GaussianBlur(img,(5,5),0) # 0 (sigmaX => X 방향의 가우스 커널 표준 편차)

# Median Blur
# kernel window와 pixel의 값들을 정렬한 후에 중간값을 선택하여 적용합니다. 
# salt-and-pepper noise(점잡음이 발생한 이미지에는 무작위적인 희고 검은 점이 나타나게 된다.) 제거에 가장 효과적
dst3 = cv2.medianBlur(img,9) # 9 (ksize – 1보다 큰 홀수)

# Bilateral Filtering
# 지금까지의 Blur처리는 경계선까지 Blur처리가 되어, 경계선이 흐려지게 됩니다. 
# #Bilateral Filtering(양방향 필터)은 경계선을 유지하면서 Gaussian Blur처리를 해주는 방법입니다.
# Gaussian 필터를 적용하고, 또 하나의 Gaussian 필터를 주변 pixel까지 고려하여 적용하는 방식
dst4 = cv2.bilateralFilter(img,9,75,75)
'''
cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)
Parameters:	
src – 8-bit, 1 or 3 Channel image
d – filtering시 고려할 주변 pixel 지름
sigmaColor – Color를 고려할 공간. 숫자가 크면 멀리 있는 색도 고려함.
sigmaSpace – 숫자가 크면 멀리 있는 pixel도 고려함.
'''

images = [img,dst1,dst2,dst3,dst4]
titles=['Original','Blur(7X7)','Gaussian Blur(5X5)','Median Blur','Bilateral']

for i in range(5):
    plt.subplot(3,2,i+1),plt.imshow(images[i]),plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()

In [ ]:
#image_contours
'''
Contours란 동일한 색 또는 동일한 강도를 가지고 있는 영역의 경계선을 연결한 선
'''
#-*- coding:utf-8 -*-
import cv2
import numpy as np

img = cv2.imread('img/earth.png')
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#threshold를 이용하여 binary image로 변환
ret,thresh = cv2.threshold(imgray,127,255,0)
#contours는 point의 list형태. 예제에서는 사각형이 하나의 contours line을 구성하기 때문에 
#len(contours) = 1. 값은 사각형의 꼭지점 좌표.
#hierachy는 contours line의 계층 구조
_, contours, _ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(img, contours, -1, (0,0,255), 1)

cv2.imshow('thresh', thresh)
cv2.imshow('contour', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# image_filtering
'''
일반적으로 고주파는 밝기의 변화가 많은 곳, 즉 경계선 영역에서 나타나며, 일반적인 배경은 저주파 
이것을 바탕으로 고주파를 제거하면 Blur처리가 되며, 저주파를 제거하면 대상의 영역을 확인
Low-pass filter(LPF)와 High-pass filter(HPF)를 이용하여, 
LPF를 적용하면 노이즈제거나 blur처리를 할 수 있으며, HPF를 적용하면 경계선을 찾음
'''

#-*- coding:utf-8 -*-
import cv2
import numpy as np

def nothing(x):
    pass

img = cv2.imread('img/snoopy.jpg')

cv2.namedWindow('image')
cv2.createTrackbar('K','image',1,20, nothing)

while(1):
    if cv2.waitKey(1) & 0xFF == 27:
        break
    k = cv2.getTrackbarPos('K','image')

    #(0,0)이면 에러가 발생함으로 1로 치환
    if k == 0:
        k = 1

    # trackbar에 의해서 (1,1) ~ (20,20) kernel생성
    kernel = np.ones((k,k),np.float32)/(k*2) 
    # kernel은 행렬을 의미하는데 kernel의 크기가 크면 이미지 전체가 blur처리가 많이 됩니다
    dst = cv2.filter2D(img,-1,kernel) # 이미지에 kernel(filter)를 적용하여 이미지를 Filtering

    cv2.imshow('image',dst)

cv2.destroyAllWindows()

In [ ]:
# image_gradients
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/dave.jpg')

canny = cv2.Canny(img,30,70)
'''
Canny Edge Detection
Noise Reduction
이미지의 Noise를 제거
'''
laplacian = cv2.Laplacian(img,cv2.CV_64F)
# laplacian = cv2.Laplacian(img,cv2.CV_8U)
# 이미지의 가로와 세로에 대한 Gradient를 2차 미분한 값
# (dx와 dy가 2인 경우) blob(주위의 pixel과 확연한 picel차이를 나타내는 덩어리)검출에 많이 사용
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
# Sobel & Scharr Filter
# Gaussian smoothing과 미분을 이용한 방법입니다. 
# 그래서 노이즈가 있는 이미지에 적용하면 좋습니다. 
# X축과 Y축을 미분하는 방법으로 경계값을 계산

images = [img,laplacian, sobelx, sobely, canny]
titles = ['Origianl', 'Laplacian', 'Sobel X', 'Sobel Y','Canny']

for i in range(5):
    plt.subplot(2,3,i+1),plt.imshow(images[i]),plt.title([titles[i]])
    plt.xticks([]),plt.yticks([])

plt.show()

In [ ]:
# image_Inpainting
'''
오래된 사진의 작은 흡집 등을 제거하는 방법
흠집에 위치한 픽셀을 그 이웃 픽셀을 가져와 교체
'''
import numpy as np
import cv2
img = cv2.imread('img/input.png')
mask = cv2.imread('img/mask.png',0)
dst = cv2.inpaint(img,mask,3,cv2.INPAINT_TELEA)
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()